In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 64.7 MB/s eta 0:00:00


# Вариант №1 Решение Михаила, с подсказкой от Анастасии

In [ ]:
import numpy as np
import cvxpy

# Задаем матрицу A, где A[i, j] - затраты на выполнение задачи i исполнителем j
A = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Определяем бинарные переменные
x = cvxpy.Variable(shape=(5, 5), boolean=True)

# Определяем переменную y, которая является транспонированной матрицей x
y = x.T

# Определяем ограничения
x_0 = cvxpy.sum(x[0]) >= 1
x_1 = cvxpy.sum(x[1]) >= 1
x_2 = cvxpy.sum(x[2]) >= 1
x_3 = cvxpy.sum(x[3]) >= 1
x_4 = cvxpy.sum(x[4]) >= 1

# Каждый исполнитель должен быть назначен хотя бы на одну задачу
# Сумма элементов каждого столбца матрицы y должна быть больше или равна 1
z0 = cvxpy.sum(y[0]) >= 1
z1 = cvxpy.sum(y[1]) >= 1
z2 = cvxpy.sum(y[2]) >= 1
z3 = cvxpy.sum(y[3]) >= 1
z4 = cvxpy.sum(y[4]) >= 1

# Определяем целевую функцию, которую нужно минимизировать
# Суммарные затраты на работы
total_value = cvxpy.sum(cvxpy.multiply(x, A))

# Определяем задачу оптимизации
problem = cvxpy.Problem(
    cvxpy.Minimize(total_value),
    constraints=[
        cvxpy.sum(x, axis=0) == np.ones(5),
        cvxpy.sum(x, axis=1) == np.ones(5)
    ]
)

# Решаем задачу оптимизации
result = problem.solve()

# Выводим оптимальное значение функции (минимальные затраты)
print(result)

# Получаем значения переменных x после решения задачи
x_values = x.value

# Выводим значения переменных x
print(x_values)


32.0
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


# Вариант №2 - Доработанное мною решение Михаила и Анастасии, с ипользованием циклов

In [ ]:
import numpy as np
import cvxpy

# Задаем матрицу A, где A[i, j] - затраты на выполнение задачи i исполнителем j
A = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Определяем бинарные переменные
x = cvxpy.Variable(shape=(5, 5), boolean=True)

# Определяем переменную y, которая является транспонированной матрицей x
y = x.T

# Определяем ограничения
constraints_x = [cvxpy.sum(x[i]) >= 1 for i in range(5)]
constraints_y = [cvxpy.sum(y[i]) >= 1 for i in range(5)]
additional_constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5)
]

# Определяем целевую функцию, которую нужно минимизировать
# Суммарные затраты на работы
total_value = cvxpy.sum(cvxpy.multiply(x, A))

# Определяем задачу оптимизации
problem = cvxpy.Problem(
    cvxpy.Minimize(total_value),
    constraints=[
        cvxpy.sum(x, axis=0) == np.ones(5),
        cvxpy.sum(x, axis=1) == np.ones(5)
    ]
)

# Решаем задачу оптимизации
result = problem.solve()

# Выводим оптимальное значение функции (минимальные затраты)
print(result)

# Получаем значения переменных x после решения задачи
x_values = x.value

# Выводим значения переменных x
print(x_values)



32.0
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


# Вариант №3 Решение с помощью модуля PuLP

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 67.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from pulp import *

# Задаем матрицу A, где A[i, j] - затраты на выполнение задачи i исполнителем j
A = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Создаем проблему линейного программирования
prob = LpProblem("Task_Allocation", LpMinimize)

# Создаем переменные решения
x = LpVariable.dicts(
    "x",
    [(i, j) for i in range(5) for j in range(5)],
    cat=LpBinary
)

# Определяем целевую функцию, которую нужно минимизировать
total_value = lpSum([A[i, j] * x[(i, j)] for i in range(5) for j in range(5)])
prob += total_value

# Определяем ограничения

# Каждая задача должна быть выполнена хотя бы одним исполнителем
for i in range(5):
    prob += lpSum([x[(i, j)] for j in range(5)]) >= 1

# Каждый исполнитель должен быть назначен хотя бы на одну задачу
for j in range(5):
    prob += lpSum([x[(i, j)] for i in range(5)]) >= 1

# Дополнительные ограничения
prob += lpSum([x[(i, j)] for i in range(5) for j in range(5)]) == 5  # Сумма всех переменных должна быть равна 5
for i in range(5):
    prob += lpSum([x[(i, j)] for j in range(5)]) <= 2  # Каждая задача не может быть назначена более чем двум исполнителям
for j in range(5):
    prob += lpSum([x[(i, j)] for i in range(5)]) <= 2  # Каждый исполнитель не может быть назначен более чем двум задачам

# Решаем задачу линейного программирования
prob.solve()

# Выводим оптимальное значение функции (минимальные затраты)
print("Total Value:", value(prob.objective))


Total Value: 32.0


# Вариант №4 Решение с помощью модуля SciPY и циклов

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Задаем матрицу с, где с[i, j] - затраты на выполнение задачи i исполнителем j
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Количество переменных
n = c.shape[0]

# Матрица A_eq для ограничений суммы по строкам и столбцам
A_eq = np.zeros((2 * n, n ** 2))
for i in range(n):
    for j in range(n):
        A_eq[i, i * n + j] = 1  # Сумма по строкам
        A_eq[n + j, i * n + j] = 1  # Сумма по столбцам
b_eq = np.ones(2 * n)

# Матрица A_ub и вектор b_ub для ограничений булевых переменных
A_ub = np.zeros((n ** 2, n ** 2))
for i in range(n):
    for j in range(n):
        A_ub[i * n + j, i * n + j] = 1  # x[i, j] <= 1
        for k in range(n):
            A_ub[i * n + j, i * n + k] = -1  # x[i, j] - x[i, k] <= 0
            A_ub[i * n + j, k * n + j] = -1  # x[i, j] - x[k, j] <= 0
b_ub = np.zeros(n ** 2)

# Границы для переменных x
bounds = [(0, 1)] * (n ** 2)

# Решение задачи линейного программирования
result = linprog(c.flatten(), A_ub=A_ub, b_ub=b_ub,
                 A_eq=A_eq, b_eq=b_eq, bounds=bounds)
x_values = np.round(result.x).reshape((n, n))

print("Результат оптимизации:")
print("Статус решения:", result.message)
print("Оптимальное значение:", result.fun)
print("Оптимальные значения переменных x:")
print(x_values)


Результат оптимизации:
Статус решения: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Оптимальное значение: 32.0
Оптимальные значения переменных x:
[[ 0.  0.  1.  0. -0.]
 [ 0.  0. -0.  1.  0.]
 [-0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0. -0.  0.]]


# Вариант №5 Решение с помощью модуля SciPy и Numpy вместо циклов

In [ ]:
import numpy as np
from scipy.optimize import linprog

# Задаем матрицу с, где с[i, j] - затраты на выполнение задачи i исполнителем j
c = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Количество переменных
n = c.shape[0]

# Матрица A_eq для ограничений суммы по строкам и столбцам
A_eq = np.vstack((np.kron(np.eye(n), np.ones((1, n))),
                  np.kron(np.ones((1, n)), np.eye(n))))
b_eq = np.ones(2 * n)

# Матрица A_ub и вектор b_ub для ограничений булевых переменных
A_ub = np.eye(n ** 2)
A_ub -= np.kron(np.eye(n), np.ones((n, n)))
A_ub -= np.kron(np.ones((n, n)), np.eye(n))
b_ub = np.zeros(n ** 2)

bounds = [(0, 1)] * (n ** 2)

# Решение задачи оптимизации
result = linprog(c.flatten(), A_ub=A_ub, b_ub=b_ub,
                 A_eq=A_eq, b_eq=b_eq, bounds=bounds)

if result.success:
    x_values = np.round(result.x).reshape((n, n))
    print("Результат оптимизации:")
    print("Статус решения:", result.message)
    print("Оптимальное значение:", result.fun)
    print("Оптимальные значения переменных x:")
    print(x_values)
else:
    print("Ошибка в оптимизации:", result.message)


Результат оптимизации:
Статус решения: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Оптимальное значение: 32.0
Оптимальные значения переменных x:
[[ 0.  0.  1.  0. -0.]
 [ 0.  0. -0.  1.  0.]
 [-0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 1.  0.  0. -0.  0.]]


# Вариант #6 Совместное использование SciPy & PuLP (Mix and Match)

In [ ]:
import numpy as np
from scipy.optimize import minimize
import pulp

# Задаем матрицу A
A = np.array([[1000, 12, 10, 19, 8],
              [12, 1000, 3, 7, 2],
              [10, 3, 1000, 6, 20],
              [19, 7, 6, 1000, 4],
              [8, 2, 20, 4, 1000]])

# Создаем модель PuLP
model = pulp.LpProblem("Assignment Problem", pulp.LpMinimize)

# Определяем переменные решения
variable_range = [(i, j) for i in range(5) for j in range(5)]
x = pulp.LpVariable.dicts("x", variable_range, cat="Binary")

# Определяем целевую функцию
model += pulp.lpSum(A[i, j] * x[(i, j)] for i in range(5) for j in range(5))

# Определяем ограничения
for i in range(5):
    model += pulp.lpSum(x[(i, j)] for j in range(5)) == 1
    model += pulp.lpSum(x[(j, i)] for j in range(5)) == 1

# Решаем модель PuLP
model.solve()

# Получаем значения переменных x из решения
x_values = np.array([[pulp.value(x[(i, j)]) for j in range(5)] 
                     for i in range(5)])

# Преобразуем значения переменных x в одномерный массив 
# для использования в scipy.optimize.minimize
x0 = x_values.flatten()

# Определяем функцию для оптимизации
def objective(x):
    x = np.reshape(x, (5, 5))
    return np.sum(np.multiply(x, A))

# Определяем ограничения
def constraint(x):
    x = np.reshape(x, (5, 5))
    constraints = []
    constraints.append(np.sum(x, axis=0) - 1)
    constraints.append(np.sum(x, axis=1) - 1)
    return constraints

# Определяем границы переменных (0 или 1)
bounds = [(0, 1)] * 25

# Определяем ограничения в виде словаря
constraints = [{'type': 'eq', 'fun': lambda x: c} for c in constraint(x0)]

# Решаем задачу оптимизации
result = minimize(objective, x0, method='SLSQP',
                  bounds=bounds, constraints=constraints)

# Выводим оптимальное значение функции (минимальные затраты)
print("Total Value:", result.fun)

# Получаем значения переменных x после решения задачи
x_values = np.reshape(result.x, (5, 5))

# Выводим значения переменных x
print(x_values)


Total Value: 32.0
[[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]
